> Dana jest tablica $ A $ zawierająca $ n $ elementów, z których każdy ma jeden z k kolorów. Proszę podać możliwie jak najszybszy algorytm, który znajduje indeksy $ i $ oraz $ j $ takie, że wsród elementów $ A[i] $, $ A[i + 1] $, $ ... $ ,$ A[j] $ występują wszystkie $ k $ kolorów oraz wartość $ j − i $ jest minimalna (innymi słowy, szukamy najkrótszego przedziału z wszystkimi kolorami).

### Omówienie algorytmu

###### Zliczanie kolorów w przedziałach

Ponieważ mamy jawnie daną liczbę różnych kolorów, wynoszącą $ k $, możemy w łatwy sposób sprawdzać, czy dany podciąg tablicy kolorów spełnia warunek taki, że występują w nim wszystkie kolory. Idea jest taka, że będziemy przeglądać tablicę liniowo, przy pomocy dwóch wskaźników, z których pierwszy będzie wyznaczał obecnie sprawdzany element i za razem początek bieżącego podciągu, a drugi koniec tego podciągu. Musimy jeszcze znaleźć taktykę na odpowiednie poruszanie się tymi wskaźnikami oraz wydajne (najlepiej w czasie $ O(1) $) sprawdzanie, czy dany kolor już został uwzględniony. Oczywiście użycie słownika lub zbioru Pythonowego jest zabronione, a to znacznie by nam ułatwiło sprawdzenie. Musimy się więc posłużyć dodatkową tablicą, która będzie przechowywała liczby wystąpień poszczególnych kolorów pod odpowiadającymi danym kolorom indeksami. Zauważmy, że wciąż nie jest to wystarczające, ponieważ chcemy wiedzieć, ile różnych kolorów w tej tablicy się znajduje. Oczywiście można by za każdym razem przebiec całą tablicę liczników kolorów i sprawdzić, czy każdy licznik jest niezerowy (każdy kolor ma przynajmniej jedno wystąpienie), ale nie jest to wydajne rozwiązanie, bo otrzymalibyśmy złożoność obliczeniową całego algorytmu wynoszącą $ O(n \cdot k) $, a możemy rozwiązać ten problem w czasie $ O(n) $. Wystarczy utworzyć dodatkową zmienną, która będzie zliczała różne kolory w danym przedziale. Zauważmy, że będziemy łatwo w stanie ocenić, czy dany kolor już wystąpił w obecnie sprawdzanym przedziale, ponieważ wystarczy sprawdzić licznik wystąpień bieżącego koloru w tablicy z licznikami (dostęp do odpowiedniej komórki tablicy ma złożoność $ O(1) $) i, jeżeli licznik ten wynosi $ 0 $, inkrementować wartość zmiennej zliczającej liczbę różnych kolorów w przedziale.

###### Przesuwanie wskaźników

Musimy jeszcze przemyśleć, jak i kiedy należy poruszyć dany wskaźnik. Ponieważ szukamy przedziału, który ma dokładnie $ k $ ($ k $ to liczba wszystkich kolorów, ale jest dana) różnych kolorów, wystarczy w pętli, odpowiadającej za przemieszczanie pierwszego wskaźnika, sprawdzać, czy zmienna, zliczająca różne kolory (omówiona wyżej), osiągnęła pożądaną wartość równą $ k $. Jeżeli tak, wystarczy sprawdzić, czy długość bieżącego podciągu jest mniejsza od poprzednio zapamiętanej największej długości i zaktualizować tę długość i indeksy najkrótszego przedziału, jeżeli bieżący podciąg jest krótszy od poprzednio zapamiętanego. Pozostaje jeszcze kwestia przemieszczania drugiego wskaźnika. Tutaj najlepiej przyjąć zasadę, że ten wskaźnik możemy ruszyć tylko wtedy, gdy znajduje się on na polu, odpowiadającemu takiemu kolorowi, jak pole, na którym umieszczony został właśnie pierwszy wskaźnik (ponieważ nie potrzebujemy z dwóch stron podciągu tych samych kolorów, tego ostatniego można się pozbyć). Tutaj jeszcze jedna uwaga, a mianowicie w wielu przypadkach przemieszczenie drugiego wskaźnika tylko o jedno pole w prawo może okazać się niewystarczające, bo w bieżącym przedziale wciąż są powtórzone kolory, których możemy się pozbyć (tzn. ten nowo dodany kolor spowodował, że możemy odrzucić nie tylko ten kolor z końca podciągu, ale również kilka takich kolorów, które się powtarzały w tym pociągu). Problem rozwiązuje użycie pętli $ while $ zamiast instrukcji warunkowej i przemieszczanie drugiego wskaźnika do momentu, w którym znajdzie się on na takim kolorze, który występuje w danym przedziale dokładnie jeden raz.

###### Drobna uwaga na koniec

Warto sobie zdać sprawę, że po osiągnięciu przez naszą zmienną, zliczającą liczbę różnych kolorów w bieżącym podciągu, wartości równej $ k $, wartość tej zmiennej już nigdy nie zostanie zmieniona w żadną stronę, ponieważ zawsze będziemy tak postępować, by nasz przedział zawierał wymaganą liczbę kolorów i odrzucać tylko te kolory, które się powtarzają. Dobrym pomysłem może się więc okazać niesprawdzanie warunku, czy osiągnęliśmy odpowiednią liczbę kolorów w przedziale w każdej pęti. Znacznie lepiej utworzyć osobno pierwszą pętlę, która znajdzie nam pierwszy przedział, w którym występuje dokładnie $ k $ różnych wartości, a następnie już nie sprawdzać tego warunku, a jedynie zmieniać liczby wystąpień odpowiednich kolorów w tablicy z ich licznikami.

##### Funkcja testująca poprawność algorytmu

In [1]:
import random

def test(fn: 'functions which is tested',
         k: 'number of unique colors in a subsequence' = None,  # When set to None colors_count will be sued as k
         colors_count: 'number of all unique colors' = 5,
         arr_length: 'number of array elements (colors)' = (0, 100),
         samples: 'number of test repetitions' = 20,
         failed_only: 'flag to show or hide output' = False
        ):
    passed = 0
    # Assign colors_count value to k if k is set to None
    k = k or colors_count
    for test_num in range(1, samples + 1):
        # Generate a random array of colors (numbers from 0 to colors_count-1)
        arr = [random.randint(0, colors_count-1) for _ in range(random.randint(*arr_length)
                                        if not isinstance(arr_length, int) else arr_length)]
        if len(arr) < k:
            expected = {(None, None)}
        else:
            # Search for the expected indices of a sequence (this algorithm is simple
            # but it runs really slow)
            expected = set()
            min_length = len(arr)
            for i in range(len(arr) - k + 1):
                for j in range(i + k, len(arr) + 1):
                    if len(set(arr[i:j])) == k:
                        curr_length = j - i
                        if curr_length < min_length:
                            expected = {(i, j - 1)}
                            min_length = curr_length
                        elif curr_length == min_length:
                            expected.add((i, j - 1))
                        break
            if not expected:
                expected.add((None, None))
        # Test if an output of our function is correct
        result = fn(arr, k, colors_count)
        is_correct = result in expected
        passed += is_correct
        # Show results if not turned off
        if not failed_only or (failed_only and not is_correct):
            print(f"TEST #{test_num}:")
            print(f"Result: i = {result[0]}, j = {result[1]}")
            print(f"Possible solutions: \n (i, j) ⋳ {expected}")
            print(f"Number of unique colors searched: {k}")
            print(f"Number of unique colors in an array: {colors_count}")
            print(f"Test {'PASSED' if is_correct else 'FAILED'}")
            print(f"Current passed-to-test ratio: {passed}/{test_num}")
            print(f"Input array:")
            print(arr)
            print()
            
    # Print the final results
    print(f"===== Final results: =====")
    print(f"Total tests passed: {passed}/{samples}")
    print(f"An algorithm is {'CORRECT' if passed == samples else 'WRONG'}")

### Implementacja algorytmu

In [2]:
# I generalize this problem to any k colors value in a subsequence
# I assume that colors are represented by positive integers from 0 to colors_count-1 
def shortest_k_colors_subsequence(arr: 'array of colors represented by numbers', 
                                  k: 'number of unique colors in a subsequence',
                                  colors_count: 'number of all unique colors'):
    # Return None if a number of unique colors is too small
    if k < 1 or not arr: return None, None
    # Initialize a counters array in order to count occurrences of a color
    # within the currently checked range
    counters = [0] * colors_count
    
    # Find the first matching subsequence
    i, j = find_first_subseq(arr, k, counters)
    # Return None if not found
    if i is None: return None, None
    
    # As we found the first matching subsequence of k unique colors, we can
    # begin searching for a possibly shorter one
    min_length = j - i + 1
    min_i = i
    min_j = j
    for j in range(j + 1, len(arr)):
        counters[arr[j]] += 1
        # Move the j pointer till we have repeated colors at the enf of
        # our subsequence (use this if and while block in order to check
        # if a new subsequence is longer than the previous one only if we
        # moved the j pointer (doing so after moving the i pointer is pointless))
        if counters[arr[i]] > 1:
            counters[arr[i]] -= 1
            i += 1
            while counters[arr[i]] > 1:
                counters[arr[i]] -= 1
                i += 1
            # Check if a new subsequence is shorter than the previous one
            curr_length = j - i + 1
            if curr_length < min_length:
                min_length = curr_length
                min_i = i
                min_j = j
                
    # Return indices of the shorter subsequence which contains all colors
    return min_i, min_j


def find_first_subseq(arr, k, counters):
    # Search fo the first subsequence which contains k different colors
    unique_colors = 0
    for j in range(len(arr)):
        counters[arr[j]] += 1
        if counters[arr[j]] == 1:
            unique_colors += 1
            if unique_colors == k:
                break
    else:
        # If a for loop above hasn't been broken, return None as we are sure
        # that unique_colors value is different to the k value
        return None, None
    
    i = 0
    # Move the second pointer till we can drop repeated colors
    while counters[arr[i]] > 1:
        counters[arr[i]] -= 1
        i += 1
        
    return i, j

###### Kilka testów

In [3]:
test(shortest_k_colors_subsequence, samples=1000, failed_only=True)

===== Final results: =====
Total tests passed: 1000/1000
An algorithm is CORRECT
